In [1]:
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
TIME_STEPS = 28     # same as the height of the image 每次读取一行，需要读取28次，所以是28
INPUT_SIZE = 28     # same as the width of the image 每一行读取多少个像素点
BATCH_SIZE = 50     #每次训练多少个图片
BATCH_INDEX = 0
OUTPUT_SIZE = 10    #读完每张图片以后，输出的结果
CELL_SIZE = 50      #相当于一个hidden layer，RNN里面要放多少个hidden unit
LR = 0.001          #learning rate

In [3]:
# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# data pre-processing
X_train = X_train.reshape(-1, 28, 28) / 255.      # normalize，变成0-1的区间
X_test = X_test.reshape(-1, 28, 28) / 255.        # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [4]:
# build RNN model
model = Sequential()

# RNN cell
model.add(SimpleRNN(
    # for batch_input_shape, if using tensorflow as the backend, we have to put None for the batch_size.
    # Otherwise, model.evaluate() will get error.
    batch_input_shape=(None, TIME_STEPS, INPUT_SIZE),       # Or: input_dim=INPUT_SIZE, input_length=TIME_STEPS,视频里none的部分是BATCH_SIZE
    output_dim=CELL_SIZE,
    unroll=True,
))

# output layer 接收的就是cellsize的个数
model.add(Dense(OUTPUT_SIZE))   # 输进全连接层输出10个节点
model.add(Activation('softmax'))#默认的是tanh

# optimizer
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# training
for step in range(4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]#用batchindex一批一批截取，一次截取50张，后面是28个步长，28个inputsize
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]#y是2维，所以少一个：
    cost = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX#如果循环到底，即这个数大于等于整个的shape了，就从头再来

    if step % 500 == 0:
        cost, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], verbose=False)
        print('test cost: ', cost, 'test accuracy: ', accuracy)

C:\Users\zhang\AppData\Local\Continuum\anaconda3\envs\tensorflow-sessions\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(batch_input_shape=(None, 28,..., unroll=True, units=50)`
  # Remove the CWD from sys.path while we load stuff.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where








test cost:  2.3910253047943115 test accuracy:  0.08160000294446945
test cost:  0.6196067929267883 test accuracy:  0.8051000237464905
test cost:  0.4972466826438904 test accuracy:  0.8472999930381775
test cost:  0.3946739733219147 test accuracy:  0.8833000063896179
test cost:  0.3616984784603119 test accuracy:  0.8971999883651733
test cost:  0.2975054979324341 test accuracy:  0.9168000221252441
test cost:  0.31962764263153076 test accuracy:  0.906000018119812
test cost:  0.24769102036952972 test accuracy:  0.9307000041007996
test cost:  0.24047143757343292 test accuracy:  0.9351999759674072
